In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [7]:
import torch
import numpy as np
import pandas as np
import pickle
from tqdm import tqdm
import sys

In [3]:
deck1 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck11.pth")
deck2 = torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-deck2.pth")

In [4]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([152791, 768]), torch.Size([1418916, 768]))

In [5]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [10]:
N = deck2["feat"].size(0)
distdeck = {"dist": [], "level": []}
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][i, None], p=None, dim=1)
        dist_, indices_ = dist_.topk(4, largest=False)
        tlevels = deck1["tlevel"][indices_]

        distdeck["dist"].append(dist_.cpu())
        distdeck["level"].append(tlevels)

        t.update()

100%|████████████████████████████████████████████████████| 1418916/1418916 [54:50<00:00, 431.19it/s]


In [14]:
distdeck["dist"] = torch.stack(distdeck["dist"])
distdeck["level"] = torch.stack(distdeck["level"])

In [15]:
torch.save(distdeck, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-distdeck.pth")

In [11]:
# 유사도 분포를 그려보기